# PPO

In [39]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/EVA_MiLab_Hackathon

%load_ext autoreload
%autoreload 2

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/EVA_MiLab_Hackathon
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
!pip install swig
!pip install "gymnasium[box2d]"

In [41]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [44]:
import gymnasium as gym
import numpy as np
from matplotlib import pyplot as plt
from agents.cnn_ppo import CNN_PPO_Agent


def train_agent(episodes=500):
    env = gym.make("CarRacing-v3", render_mode="rgb_array", lap_complete_percent=0.95, domain_randomize=False, continuous=True)

    agent = CNN_PPO_Agent(input_shape=(3, 96, 96))
    agent.load_checkpoint()

    for episode in range(episodes):
        state, info = env.reset()

        # Convert state to Tensor and reshape
        state = torch.tensor(state, dtype=torch.float32, device=agent.device)
        state = state.permute(2, 0, 1).unsqueeze(0)  # Fix (H, W, C) → (1, C, H, W)
        #print(f"State shape: {state.shape}")  # Debug print


        total_reward = 0
        done = False

        while not done:
            action, log_prob, value = agent.select_action(state)

            # Ensure `action` is a NumPy array before passing to env.step()
            action = np.array(action, dtype=np.float32)

            #print(f"Action: {action}, Shape: {action.shape}")  # Debug print

            next_state, reward, done, _, _ = env.step(action)

            # Convert next_state to Tensor and reshape
            next_state = torch.tensor(next_state, dtype=torch.float32, device=agent.device)
            next_state = next_state.permute(2, 0, 1).unsqueeze(0)  # Fix shape (1, 3, 96, 96)

            agent.store_transition((state, action, reward, next_state, log_prob, value, done))
            agent.update()  # Train PPO

            state = next_state
            total_reward += reward

        agent._log_reward(episode, total_reward)

        if episode % 50 == 0:
            agent.save_checkpoint(episode)

        print(f"Episode {episode}: Total Reward: {total_reward}")

    env.close()

State Before CNN: (3, 96, 96)


/content/gdrive/MyDrive/EVA_MiLab_Hackathon/agents/cnn_ppo.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Selects action using PPO policy for continuous action space"""


KeyboardInterrupt: 

In [ ]:
train_agent()